In [11]:
import json

# {'instruction': 'Can you recount a detailed memory of the first time you saw a sonogram of a baby, describing what you observed and felt during the experience?',
#  'source': 'gpt4o',
#  'type': 'Literary Fiction',
#  'postedit': 'The soft hum of machinery filled the room with an unfamiliar yet gentle melody. I sat beside Lila, squeezing her hand as she lay on the table while the technician swirled the wand over her rounded belly. The screen beside us flickered to life, a grainy black and white. Slowly, an image began to form; the unmistakable curve of a tiny head, the flutter of something I later learned was a heartbeat. I leaned forward, almost holding my breath, mesmerized by the play of light and shadows that composed this new life. Lila\'s grip tightened and upon glancing at her, I found her eyes glossy, along with a smile tugging at the corners of her mouth. I turned back to the screen, emotions swirling—awe, fear, hope. In that moment, the abstract idea of "our baby" transformed into something real, tangible. The room\'s dimness seemed to fold around us, making space for this silent, wordless connection. There, in the ghostly shapes on the monitor, I saw not just a child, but a future.',
#  'id': 'W18_batch2',
#  'creativity_scores': ['5', '7'],
#  'fine_grained_edits': [{'categorization': 'Awkward Word Choice and Phrasing',
#    'originalText': 'The room was dimly lit, with the soft hum of machinery filling the silence',
#    'editedText': 'The soft hum of machinery filled the room with an unfamiliar yet gentle melody.'},
#   {'categorization': 'Lack of Specificity and Detail',
#    'originalText': 'I sat beside Lila, squeezing her hand, as the technician swirled the wand over her belly',
#    'editedText': 'I sat beside Lila, squeezing her hand as she lay on the table while the technician swirled the wand over her rounded belly.'},
#   {'categorization': 'Unnecessary Exposition',
#    'originalText': "The screen flickered to life, a grainy black and white, like an ancient TV trying to find it's signal",
#    'editedText': 'The screen beside us flickered to life, a grainy black and white.'},
#   {'categorization': 'Poor Sentence Structure',
#    'originalText': "Lila's grip tightened, and I glanced at her, finding her eyes glossy, a smile tugging at the corners of her mouth",
#    'editedText': "Lila's grip tightened and upon glancing at her, I found her eyes glossy, along with a smile tugging at the corners of her mouth."},
#   {'categorization': 'Unnecessary Exposition',
#    'originalText': '—a future that felt both incredibly close and impossibly distant',
#    'editedText': ''}],
#  'data-split': 'train',
#  'preedit': 'The room was dimly lit, with the soft hum of machinery filling the silence. I sat beside Lila, squeezing her hand, as the technician swirled the wand over her belly. The screen flickered to life, a grainy black and white, like an ancient TV trying to find it\'s signal. Slowly, an image began to form; the unmistakable curve of a tiny head, the flutter of something I later learned was a heartbeat. I leaned forward, almost holding my breath, mesmerized by the play of light and shadows that composed this new life. Lila\'s grip tightened, and I glanced at her, finding her eyes glossy, a smile tugging at the corners of her mouth. I turned back to the screen, emotions swirling—awe, fear, hope. In that moment, the abstract idea of "our baby" transformed into something real, tangible. The room\'s dimness seemed to fold around us, making space for this silent, wordless connection. There, in the ghostly shapes on the monitor, I saw not just a child, but a future—a future that felt both incredibly close and impossibly distant.',
#  'editor': 'W18',
#  'creativity_pre_score': 5,
#  'creativity_post_score': 7,
#  'creativity_z_score_pre': 3.6285013258392107,
#  'creativity_z_score_post': 5.328722422859342,
#  'creativity_z_score_pre_int': 4,
#  'creativity_z_score_post_int': 5,
#  'creativity_z_score_diff': 1,
#  'editor_split': 'test'}

with open("prompts/lamp_cot_input.txt", "r") as f:
    lamp_cot_input_prompt = f.read()

with open("prompts/lamp_cot_output.txt", "r") as f:
    lamp_cot_output_prompt = f.read()

with open("prompts/lamp_detection_input.txt", "r") as f:
    lamp_detection_input_prompt = f.read()

with open("prompts/lamp_rewriting_input.txt", "r") as f:
    lamp_rewriting_input_prompt = f.read()

def prepare_cot_sample(sample):
    input_text = lamp_cot_input_prompt.replace("[[INPUT_PARAGRAPH]]", sample["preedit"])

    problematic_spans_STR = """"""
    for i, edit in enumerate(sample["fine_grained_edits"]):
        problematic_spans_STR += f"Span {i+1}: `{edit['originalText']}` (Category: `{edit['categorization']}`)\n"

    proposed_rewriting_STR = """"""
    for i, edit in enumerate(sample["fine_grained_edits"]):
        proposed_rewriting_STR += f"Span {i+1}: `{edit['originalText']}` -> `{edit['editedText']}`\n"

    output_text = lamp_cot_output_prompt.replace("[[PROBLEMATIC_SPANS]]", problematic_spans_STR).replace("[[PROPOSED_REWRITING]]", proposed_rewriting_STR).replace("[[FINAL_PARAGRAPH]]", sample["postedit"])
    return {"id": sample["id"], "input_text": input_text, "output_text": output_text}

def prepare_detection_sample(sample):
    input_text = lamp_detection_input_prompt.replace("[[INPUT_PARAGRAPH]]", sample["preedit"])
    output_text = json.dumps({"edits": [{"span": edit["originalText"], "category": edit["categorization"]} for edit in sample["fine_grained_edits"]]})
    return {"id": sample["id"], "input_text": input_text, "output_text": output_text}

def prepare_rewriting_samples(sample):
    samples = []
    for edit_idx, edit in enumerate(sample["fine_grained_edits"]):
        input_text = lamp_rewriting_input_prompt.replace("[[INPUT_PARAGRAPH]]", sample["preedit"]).replace("[[PROBLEMATIC_SPAN]]", edit["originalText"]).replace("[[PROBLEMATIC_CATEGORY]]", edit["categorization"])
        output_text = json.dumps({"rewrite": edit["editedText"]})
        samples.append({"id": f"{sample['id']}_edit_{edit_idx}", "input_text": input_text, "output_text": output_text})
    return samples

with open("data/LAMP-train-val-test.json", "r") as f:
    lamp_data = json.load(f)

lamp_train = [d for d in lamp_data if d["data-split"] == "train"]

with open("data/lamp_train_cot_format.json", "w") as f:
    json.dump([prepare_cot_sample(sample) for sample in lamp_train], f, indent=4)

with open("data/lamp_train_detection_format.json", "w") as f:
    json.dump([prepare_detection_sample(sample) for sample in lamp_train], f, indent=4)

lamp_rewriting_samples = []
for sample in lamp_train:
    lamp_rewriting_samples += prepare_rewriting_samples(sample)

with open("data/lamp_train_rewriting_format.json", "w") as f:
    json.dump(lamp_rewriting_samples, f, indent=4)

print("Created the three datasets.")
print(f"COT has {len(lamp_train)} samples.")
print(f"Detection has {len(lamp_train)} samples.")
print(f"Rewriting has {len(lamp_rewriting_samples)} samples.")

Created the three datasets.
COT has 1000 samples.
Detection has 1000 samples.
Rewriting has 7234 samples.


In [12]:
# load cot dataset
with open("data/lamp_train_cot_format.json", "r") as f:
    lamp_train_cot = json.load(f)

print(lamp_train_cot[0]["output_text"])


Part 1: Idenfying Problematic Spans

Span 1: `The room was dimly lit, with the soft hum of machinery filling the silence` (Category: `Awkward Word Choice and Phrasing`)
Span 2: `I sat beside Lila, squeezing her hand, as the technician swirled the wand over her belly` (Category: `Lack of Specificity and Detail`)
Span 3: `The screen flickered to life, a grainy black and white, like an ancient TV trying to find it's signal` (Category: `Unnecessary Exposition`)
Span 4: `Lila's grip tightened, and I glanced at her, finding her eyes glossy, a smile tugging at the corners of her mouth` (Category: `Poor Sentence Structure`)
Span 5: `—a future that felt both incredibly close and impossibly distant` (Category: `Unnecessary Exposition`)


Part 2: Proposing Rewriting for Problematic Spans

Span 1: `The room was dimly lit, with the soft hum of machinery filling the silence` -> `The soft hum of machinery filled the room with an unfamiliar yet gentle melody.`
Span 2: `I sat beside Lila, squeezing her

## Prepare Evaluation Benchmark

In [15]:
import json

# lamp_fn = "data/LAMP-train-val-test.json"

# with open(lamp_fn, "r") as f:
#     lamp_data = json.load(f)
# lamp_test = [d for d in lamp_data if d["data-split"] == "test"]
# instruction2lamp_sample = {sample['instruction']: sample for sample in lamp_test}

gold_prefs = "data/gold_preference_600.json"
with open(gold_prefs, "r") as f:
    gold_prefs = json.load(f)

eval_samples = []
for sample_id, gold_pref in enumerate(gold_prefs):
    sample_id = f"gold_{sample_id}"
    candidates = [
        {"id": f"{sample_id}_ai_draft", "system": "ai_draft", "text": gold_pref["AI-generated"]},
        {"id": f"{sample_id}_human_edited", "system": "human_edited", "text": gold_pref["Human-edited"]},
        {"id": f"{sample_id}_fs_edited", "system": "fs_edited", "text": gold_pref["AI-edited"]},
    ]
    sample = {"id": sample_id, "instruction": gold_pref["instruction"], "candidates": candidates}
    eval_samples.append(sample)


len(eval_samples)

with open("data/lamp_editing_benchmark.json", "w") as f:
    json.dump(eval_samples, f, indent=4)


In [16]:
from llms import generate_json
import tqdm

with open("prompts/reward_calc.txt", "r") as f:
    reward_calc_prompt = f.read()

REWARD_MODEL = "gpt-4o"

def calculate_reward(candidate):
    input_text = reward_calc_prompt.replace("[[PARAGRAPH]]", candidate["text"])
    output_response = generate_json([{"role": "user", "content": reward_calc_prompt}], model=REWARD_MODEL, variables={"PARAGRAPH": candidate["text"]})
    candidate["score"] = output_response["score"]

with open("data/lamp_editing_benchmark.json", "r") as f:
    eval_samples = json.load(f)

todos = [candidate for sample in eval_samples for candidate in sample["candidates"]]
for i, todo in tqdm.tqdm(enumerate(todos)):
    # calculate_reward(todo)

    if i%10 == 0 or i == len(todos)-1:
        with open("data/lamp_editing_benchmark_with_scores.json", "w") as f:
            json.dump(eval_samples, f, indent=4)


603
